In [5]:
# 导入 sqlite3 模块，它是 Python 自带的轻量级数据库工具
# 无需额外安装，可直接操作 SQLite 数据库文件（如 test.db）
import sqlite3

In [3]:
# coding
import sqlite3
# 连接数据库
conn = sqlite3.connect("test.db")
# 创建一个游标对象，用于执行sql，获取查询结果
cursor = conn.cursor
# 员工表
cursor.execute("""
CREATE TABLE IF NOT EXISTS employees(
    id INTEGER PRIMARY KEY,
    name TEXT,
    department TEXT,
    salary INTEGER
)
""")

sample_data = [ 
    (6, "黄佳", "销售", 50000), 
    (7, "宁宁", "工程", 75000), 
    (8, "谦谦", "销售", 60000), 
    (9, "悦悦", "工程", 80000), 
    (10, "黄仁勋", "市场", 55000),
    (11, "曾繁华", "工程", 80000)
]





In [ ]:
# 打开（或创建）一个名为 "test.db" 的 SQLite 数据库文件
# 如果当前目录下没有这个文件，Python 会自动创建它
# conn 是一个“连接对象”，代表程序和数据库之间的连接
conn = sqlite3.connect("test2.db")

# 创建一个“游标”（cursor），用于执行 SQL 命令
# 可以把游标想象成数据库操作的“光标”或“遥控器”
cursor = conn.cursor()

# 使用游标执行一条 SQL 语句：创建一个名为 employees（员工）的表
# 如果该表已经存在，则不会重复创建（因为用了 IF NOT EXISTS）
# 使用三个双引号 """（或三个单引号 '''）来包裹多行字符串
cursor.execute("""
CREATE TABLE IF NOT EXISTS employees(
    id INTEGER PRIMARY KEY,     -- 员工编号，整数类型，且是主键（唯一标识每条记录）
    name TEXT,                  -- 员工姓名，文本类型
    department TEXT,            -- 所属部门，文本类型
    salary INTEGER              -- 工资，整数类型（单位可以是元/月等，根据需求定）
)
""")

# （可选但推荐）提交事务，确保建表操作真正保存到数据库中
# 虽然 CREATE TABLE 通常会自动提交，但养成好习惯很重要
# conn.commit()

# 关闭数据库连接，释放资源（就像关掉文件一样）
# conn.close()

In [7]:
# 定义一个包含多条员工信息的列表
# 每个员工用一个元组 (id, 姓名, 部门, 工资) 表示
# 注意：这里的 id 必须是唯一的（因为数据库表中 id 是主键）
sample_data = [
    (6, "张三", "开发部", 32000),
    (7, "李四", "销售部", 20000),
    (8, "王五", "开发部", 33000),
    (9, "钱六", "销售部", 15000)
]

# 使用 cursor 对象批量插入数据到 employees 表中
# 'INSERT INTO employees VALUES(?, ?, ?, ?)' 是 SQL 插入语句
#   - ? 是“占位符”，代表后面 sample_data 中的每一个值
#   - 这种写法安全，能防止 SQL 注入攻击
# executemany() 会自动遍历 sample_data 中的每一行，并逐条插入
cursor.executemany('INSERT INTO employees VALUES(?, ?, ?, ?)', sample_data)

# 提交（保存）所有刚刚插入的数据到数据库文件（test.db）
# 如果不调用 commit()，数据可能不会真正写入磁盘！
conn.commit()

In [8]:
from openai import OpenAI
client = OpenAI(
    api_key='sk-c66902a473544b04be3883e81418879d',
    base_url='https://api.deepseek.com/v1'
)

In [9]:
# 这行代码的作用是：向 SQLite 数据库发送一个特殊命令（PRAGMA table_info），
# 用来获取名为 "employees" 的表的结构信息（比如有哪些列、每列是什么类型等）。
# cursor.execute(...) 执行这个命令，fetchall() 把所有返回的结果一次性取出来，
# 结果是一个列表，列表里的每个元素代表一列的信息。
schema = cursor.execute("PRAGMA table_info(employees)").fetchall()

# 现在我们要把拿到的表结构“美化”成一条看起来像 SQL 建表语句的字符串，
# 方便人类阅读，也方便后面让大模型（比如 AI）理解数据库长什么样。

# 先写上 "CREATE TABLE EMPLOYEES (" 开头
# 然后对 schema 列表中的每一列（col）做处理：
#   - col[1] 是这一列的“列名”（比如 'id', 'name'）
#   - col[2] 是这一列的“数据类型”（比如 'INTEGER', 'TEXT'）
# 用 f"{col[1]} {col[2]}" 把它们拼成 "id INTEGER" 这样的格式
# 再用 "\n".join(...) 把所有列用换行符连接起来（每列占一行）
# 最后加上一个换行和右括号 ")"，组成完整的建表语句（虽然不能直接执行，但看起来很清晰）
schema_str = "CREATE TABLE EMPLOYEES (\n" + "\n".join([f"{col[1]} {col[2]}" for col in schema]) + "\n)"

# 打印提示文字
print("数据库Schema:")

# 打印我们刚刚构造好的表结构字符串
print(schema_str)

数据库Schema:
CREATE TABLE EMPLOYEES (
id INTEGER
name TEXT
department TEXT
salary INTEGER
)


In [16]:
# 定义一个函数，名字叫 ask_deepseek
# 它有两个输入参数：
#   - query：用户提出的问题（比如“开发部门员工的姓名和工资是多少？”）
#   - schema：数据库表的结构描述（比如有哪些列、列叫什么、是什么类型）
def ask_deepseek(query, schema):
    
    # 构造一个“提示词”（prompt），准备发给大模型（比如 DeepSeek、ChatGPT 等）
    # 使用 f-string（格式化字符串）把 schema 和 query 动态插入进去
    prompt = f"""
    这是一个数据库的Schema:
    {schema}
    根据这个Schema,你能输出一个SQL查询来回答以下问题吗？
    只输出SQL查询，不要输出任何其他内容，也不要带任何格式。
    问题:{query}
    """
    # 打印这个提示词到屏幕上（方便我们看 AI 会收到什么指令）
    # print(prompt)
    response = client.chat.completions.create(
        model="deepseek-reasoner",
        max_tokens=2048,
        messages=[
            {
                "role":"user",
                "content":prompt
            }
        ]
    )
    return response.choices[0].message.content


# 调用刚才定义的函数
# 传入两个参数：
#   - 第一个参数是用户的问题（字符串）
#   - 第二个参数是之前从数据库拿到的表结构（变量 schema）
# ask_deepseek("开发部部门员工的姓名和工资是多少？", schema)
question = "开发部部门员工的姓名和工资是多少？"
sql = ask_deepseek(question, schema_str)
print("生成的sql查询: ")
print(sql)

生成的sql查询: 
SELECT name, salary FROM EMPLOYEES WHERE department = '开发部';
